In [1]:
from ipyleaflet import (
    Map, GeoJSON, basemaps, basemap_to_tiles, Choropleth,
    LayersControl, SplitMapControl, DrawControl, MeasureControl
)
import ipywidgets as widgets
import plotly.graph_objs as go
import pandas as pd
from IPython.display import display
import json
import os
import datetime

In [2]:
def format_file_size(fsize):
    result = []
    units = {s: u for s, u in zip(reversed([2 ** n for n in range(0, 40, 10)]), ['GiB', 'MiB', 'KiB', 'bytes'])}
    for s, u in units.items():
        t = fsize // s
        if t > 0:
            result.append('{} {}'.format(t, u))
        fsize = fsize % s
    return ', '.join(result) or '0 bytes'

def file_path_to_size(file_path):
    return 'tamaño: ' + format_file_size(os.stat(file_path).st_size)

def new_remove(l, e):
    if e in l:
        l.remove(e)
        return l
    else: return l

In [3]:
base_path = os.path.join('/','srv','data')
folders = ['azero','guadalquivir','superficial']

geo_paths = {folder:os.path.join(base_path,folder,'geo') for folder in folders}
geo_file_lists = {folder:new_remove(os.listdir(geo_paths[folder]),'.ipynb_checkpoints') for folder in folders}
layer_names = {folder:[file[:-5] for file in geo_file_lists[folder]] for folder in folders}
layer_paths = {folder:[os.path.join(geo_paths[folder], file) for file in geo_file_lists[folder]] for folder in folders}
name_to_path = {folder:dict(zip(layer_names[folder], layer_paths[folder])) for folder in folders}

directories = ['calibracion','reporte']
csv_paths = [os.path.join(base_path,'superficial','csv',d) for d in directories]
csv_file_lists = [new_remove(os.listdir(path), '.ipynb_checkpoints') for path in csv_paths]
    
dfs = {}
for d,path,files in zip(directories,csv_paths,csv_file_lists):
    for file in files:
        dfs[f'{d}-{file[:-4]}']= pd.read_csv(os.path.join(path,file))

In [4]:
num_layers = 3
color_list = ['blue', 'green', 'yellow'] * 3

folder_toggle = widgets.ToggleButtons(
    options=folders,
    value=folders[0]
)

layer_checkboxes = [widgets.Checkbox(
    value=True if i==0 else False,
    layout=dict(width='100px'),
) for i in range(num_layers)]

layer_dropdowns = [widgets.Dropdown(
    options=layer_names[folder_toggle.value],
    description=f'Capa {i+1}',
    disabled= not layer_checkboxes[i].value,
) for i in range(num_layers)]

color_pickers = [widgets.ColorPicker(
    concise=True,
    value=color,
    layout=dict(width='40px')
) for color in color_list[:num_layers]]

size_labels = [widgets.Label(
    value=file_path_to_size(name_to_path[folder_toggle.value][layer_dropdowns[i].value])
) for i in range(num_layers)]

accordion = widgets.Accordion(
    selected_index=None,
    children=[
        folder_toggle,
        widgets.VBox([widgets.HBox([
            layer_checkboxes[i],
            layer_dropdowns[i],
            color_pickers[i],
            size_labels[i]
        ]) for i in range(num_layers)])
    ]
)
accordion.set_title(0,'Datos')
accordion.set_title(1,'Capas')

go_button = widgets.Button(
    description='GO!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Mostrar capa',
)

loading_label = widgets.Label()

my_map = Map(
    center=(-21.5119,-64.7253),
    zoom=7,
)
left_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
right_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
split_control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
my_map.add_control(split_control)
my_map.add_control(LayersControl())
my_map.add_control(DrawControl())
my_map.add_control(MeasureControl())

output_table = widgets.Output()
fig = go.FigureWidget(layout=dict(height=10))
fig.add_scatter()

csv_key = widgets.Label(
    description='Código:',
    layout=dict(display='none'),
)

csv_dropdown = widgets.Dropdown(
    options = list(dfs.keys()),
    layout=dict(display='none', width='450px'),
)

diagram_type = widgets.ToggleButtons(
    options=['completo','mensual'],
    value='completo',
    layout=dict(display='none')
)

In [5]:
def handle_checkbox_change(i):
    def handle_checkbox_i_change(change):
        layer_dropdowns[i].disabled = not change.new
    return handle_checkbox_i_change

def handle_layer_change(i):
    def handle_layer_i_change(change):
        file_path = name_to_path[folder_toggle.value][change.new]
        size_labels[i].value = file_path_to_size(file_path)
    return handle_layer_i_change

def handle_folder_change(change):
    for i in range(num_layers):
        layer_dropdowns[i].options = layer_names[change.new]

months = ['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']    
    
def get_scatter_x(d_type,csv):
    df = dfs[csv]
    if d_type == 'completo':
        x = [datetime.datetime(year=y, month=m, day=15) for y,m in zip(df.Year, df.Month)] 
    if d_type == 'mensual':
        x = months
    return x

def get_scatter_y(d_type,csv,code):
    df= dfs[csv]
    m_to_i = dict(zip(months,[i+1 for i in range(12)]))
    return df[code] if d_type == 'completo' else [df[df.Month == m_to_i[m]][code].mean() for m in months]
        
def handle_diagram_type_change(change):
    csv, code = csv_dropdown.value, csv_key.value
    fig.data[0].x = get_scatter_x(change.new,csv)
    fig.data[0].y = get_scatter_y(change.new,csv,code)


def click_handler(event=None, id=None, properties=None):
    output_table.clear_output()
    props = properties
    props.pop('style', None)
    with output_table:
        display(pd.DataFrame(props, index=[0]))
        
    if folder_toggle.value == 'superficial':
        code = properties['codigo']
        name = properties['nombre'] if 'nombre' in properties.keys() else properties['NOMBRE']
        csv_key.value = code
        fig.layout.height=450
        csv_dropdown.layout.display = 'block'
        diagram_type.layout.display = 'block'
        d_type, csv = diagram_type.value, csv_dropdown.value
        fig.layout.title= f'{csv}<br>{name} ({code})'
        fig.data[0].x= get_scatter_x(d_type,csv)
        fig.data[0].y= get_scatter_y(d_type,csv,code)
        
def handle_csv_change(change):
    fig.layout.title = f"{change.new}<br>{fig.layout.title.split('<br>')[1]}"
    fig.data[0].x= get_scatter_x(diagram_type.value,change.new)
    fig.data[0].y= get_scatter_y(diagram_type.value,change.new,csv_key.value)

def handle_go_click(change):
    loading_label.value = 'Cargando Capas ...'
    my_map.clear_layers()
    
    selected_layers, selected_colors = ([],[])
    for i in range(num_layers):
        if layer_checkboxes[i].value:
            selected_layers.append(layer_dropdowns[i].value)
            selected_colors.append(color_pickers[i].value)

    data_list = []
    for layer_name in selected_layers:
        with open(name_to_path[folder_toggle.value][layer_name], 'r', encoding='utf-8-sig') as f:
                data_list.append(json.load(f))
        
    geojson_layers = []
    for data, color, layer_name in zip(data_list, selected_colors, selected_layers):
        for feature in data['features']:
            feature['properties']['style'] = dict(
                color=color,
                weight=1,
                fillColor=color,
                fillOpacity=.3
            )
        
        id_to_density = {}
        if layer_name == 'densidad_poblacion_por_microcuenca':
            for feature in data['features']:
                density = feature['properties']['DENSIDAD']
                id_to_density[feature['id']] = density if density < 100 else 100 
                
            geojson_layer = Choropleth(
                geo_data=data,
                choro_data=id_to_density,
                style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
            )
        else:
            geojson_layer = GeoJSON(
                data=data,
                name=layer_name,
                hover_style={'fillColor':'red'}
            )
        geojson_layers.append(geojson_layer)
    
    for geojson_layer in geojson_layers:
        my_map.add_layer(geojson_layer)
        geojson_layer.on_click(click_handler)
    
    loading_label.value = 'Listo! Clickea en cualquier elemento para obtener más información.'

    
for i in range(num_layers):
    layer_checkboxes[i].observe(handle_checkbox_change(i),names='value')
    layer_dropdowns[i].observe(handle_layer_change(i),names='value')
    
folder_toggle.observe(handle_folder_change, names='value')
go_button.on_click(handle_go_click)
csv_dropdown.observe(handle_csv_change, names='value')
diagram_type.observe(handle_diagram_type_change, names='value')

In [6]:
widgets.VBox([
    accordion,
    widgets.HBox([go_button, loading_label]),
    my_map, 
    output_table,
    fig,csv_key, 
    widgets.HBox([csv_dropdown, diagram_type]),
])